In [36]:
import pandas as pd
from ultralytics import YOLO
from pathlib import Path 

file_location_path = Path.cwd()
project_base_path = file_location_path.parent.parent
image_folder = Path('/home/ubuntu/landscape-aesthetics')  


csv_file = project_base_path / 'data' / 'processed' / 'landscape_ob'  / 'Object_detect_from_score3_to_4.csv'
df = pd.read_csv(csv_file)


model = YOLO('yolov8n.pt')  


df['predicted_objects2'] = ''


for idx, row in df.iterrows():
    
    relative_image_path = row['image_path']  
    image_path = image_folder / relative_image_path  
    
 
    if not image_path.exists():
        print(f"Image not found: {image_path}")
        continue  


    results = model(str(image_path))  
    
    
    predictions = []
    for result in results[0].boxes:  
        class_id = int(result.cls)  
        confidence = float(result.conf)  # 获取预测置信度
        class_name = model.names[class_id]  # 使用YOLOv8的names属性获取类别名称
        predictions.append(f'{class_name}:{confidence:.2f}')
    
    # 将预测结果保存到DataFrame的新列中
    df.at[idx, 'predicted_objects2'] = '; '.join(predictions)

# 将更新后的DataFrame保存回原CSV文件中（覆盖原文件）
df.to_csv(csv_file, index=False)

print(f"The prediction results have been saved to：{csv_file}")



image 1/1 /home/ubuntu/landscape-aesthetics/data/processed/wikimedia_commons/images/UK/ns6/645/File:Kidwelly Bridge-Pont Cydweli.jpeg: 640x640 1 umbrella, 199.2ms
Speed: 3.6ms preprocess, 199.2ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/ubuntu/landscape-aesthetics/data/processed/wikimedia_commons/images/NO/ns6/2022/File:Ytteren kirkegård 20140627 IMAG0392.jpeg: 640x640 1 person, 229.4ms
Speed: 3.5ms preprocess, 229.4ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/ubuntu/landscape-aesthetics/data/processed/wikimedia_commons/images/TR/ns6/186/File:Черепаший остров недалеко от Анталии - panoramio.jpeg: 640x640 1 car, 187.9ms
Speed: 4.1ms preprocess, 187.9ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/ubuntu/landscape-aesthetics/data/processed/wikimedia_commons/images/PL/ns6/2100/File:Saint Nicholas church in Cieszyn (7).jpeg: 640x640 (no detections), 182.0ms
Speed: 3.9ms pr

In [ ]:
import pandas as pd
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt

# 设置路径
file_location_path = Path.cwd()
project_base_path = file_location_path.parent.parent
image_folder = Path('/home/ubuntu/landscape-aesthetics')  # 假设图片存储的文件夹

# 加载CSV文件
csv_file = project_base_path / 'data' / 'processed' / 'landscape_ob' / 'Object_detect_from_score2_to_3.csv'
df = pd.read_csv(csv_file)

# 获取第10到第20张图片的路径
for i in range(377, 379):  
    relative_image_path = df['image_path'].iloc[i]  # 获取第i行的图片路径
    image_path = image_folder / relative_image_path  # 拼接完整的图片路径

    # 检查图片路径是否存在
    if image_path.exists():
        # 打开并显示图片
        image = Image.open(image_path)
        plt.figure()  # 创建一个新的图像窗口
        plt.imshow(image)
        plt.axis('off')  # 关闭坐标轴显示
        plt.show()
    else:
        print(f"图片未找到: {image_path}")


In [45]:
import pandas as pd
from pathlib import Path  # 导入Path类

# 设置路径
file_location_path = Path.cwd()
project_base_path = file_location_path.parent.parent

# 加载CSV文件
csv_file = project_base_path / 'data' / 'processed' / 'landscape_ob'  / 'Object_detect_from_score5_to_6.csv'
data = pd.read_csv(csv_file)

# 定义一个函数，将物体类别和置信度解析成字典并合并
def merge_objects(row):
    obj1 = row['predicted_objects']
    obj2 = row['predicted_objects2']
    
    # 定义一个空字典存储合并后的结果
    merged_dict = {}

    # 解析并处理第一列
    if pd.notna(obj1):
        objects1 = obj1.split('; ')
        for obj in objects1:
            category, confidence = obj.split(':')
            merged_dict[category] = max(merged_dict.get(category, 0), float(confidence))  # 选择较高的置信度

    # 解析并处理第二列
    if pd.notna(obj2):
        objects2 = obj2.split('; ')
        for obj in objects2:
            category, confidence = obj.split(':')
            merged_dict[category] = max(merged_dict.get(category, 0), float(confidence))  # 选择较高的置信度
    
    # 将合并结果重新拼接为字符串形式
    merged_objects = '; '.join([f"{key}:{value:.2f}" for key, value in merged_dict.items()])
    return merged_objects

# 使用 apply 函数合并两列
data['combined_objects'] = data.apply(merge_objects, axis=1)

# 设置完整的保存路径，包括文件名
save_path = project_base_path / 'data' / 'processed' / 'landscape_ob' / 'merged_objects.csv'

# 将文件保存到指定路径
data.to_csv(save_path, index=False)

print(f"合并后的CSV文件已保存到: {save_path}")


合并后的CSV文件已保存到: /home/ubuntu/landscape-aesthetics/data/processed/landscape_ob/merged_objects.csv


In [49]:
import pandas as pd
import re
from pathlib import Path

# 设置路径
file_location_path = Path.cwd()
project_base_path = file_location_path.parent.parent

# 加载已合并的CSV文件
csv_file = project_base_path / 'data' / 'processed' / 'landscape_ob' / 'Object_detect_from_score6_to_7.csv'
data = pd.read_csv(csv_file)

# 检查列名是否正确
print(data.columns)

# 定义一个函数，去掉物体名称后面的置信度
def remove_confidence(objects_str):
    if isinstance(objects_str, str):  # 确保是字符串类型
        # 使用正则表达式匹配每个物体名称后面的 ':数字' 部分，并将其去掉
        return re.sub(r':\d+\.\d+', '', objects_str)
    else:
        return objects_str  # 如果不是字符串，直接返回原值

# 确保 'combined_objects' 列存在
if 'predicted_objects' in data.columns:
    # 对 'combined_objects' 列应用函数
    data['filtered_objects'] = data['predicted_objects'].apply(remove_confidence)

    # 保存处理后的数据为新CSV文件
    filtered_csv_file = project_base_path / 'data' / 'processed' / 'landscape_ob' / 'filtered_objects2.csv'  # 替换为保存路径
    data.to_csv(filtered_csv_file, index=False)

    print(f"过滤后的CSV文件已保存到: {filtered_csv_file}")
else:
    print("Error: 'combined_objects' 列不存在于数据中")


Index(['image_path', 'predicted_score', 'predicted_objects'], dtype='object')
过滤后的CSV文件已保存到: /home/ubuntu/landscape-aesthetics/data/processed/landscape_ob/filtered_objects2.csv


In [47]:
import pandas as pd
import re
from pathlib import Path

# 设置路径
file_location_path = Path.cwd()
project_base_path = file_location_path.parent.parent

# 加载已合并的CSV文件
csv_file = project_base_path / 'data' / 'processed' / 'landscape_ob' / 'filtered_objects2.csv'
data = pd.read_csv(csv_file)

# 检查列名是否正确
print(data.columns)

# 定义一个函数，去掉物体名称后面的置信度
def remove_confidence(objects_str):
    if isinstance(objects_str, str):  # 确保是字符串类型
        # 使用正则表达式匹配每个物体名称后面的 ':数字' 部分，并将其去掉
        return re.sub(r':\d+\.\d+', '', objects_str)
    else:
        return objects_str  # 如果不是字符串，直接返回原值

# 确保 'combined_objects' 列存在
if 'combined_objects' in data.columns:
    # 对 'combined_objects' 列应用函数
    data['filtered_objects'] = data['combined_objects'].apply(remove_confidence)

    # 删除 'predicted_objects' 和 'predicted_objects2' 列
    data = data.drop(columns=['predicted_objects', 'predicted_objects2'])

    # 保存处理后的数据为新CSV文件
    filtered_csv_file = project_base_path / 'data' / 'processed' / 'landscape_ob' / 'filtered_objects3.csv'  # 替换为保存路径
    data.to_csv(filtered_csv_file, index=False)

    print(f"过滤后的CSV文件已保存到: {filtered_csv_file}")
else:
    print("Error: 'combined_objects' 列不存在于数据中")


Index(['image_path', 'predicted_score', 'predicted_objects',
       'predicted_objects2', 'combined_objects', 'filtered_objects'],
      dtype='object')
过滤后的CSV文件已保存到: /home/ubuntu/landscape-aesthetics/data/processed/landscape_ob/filtered_objects2.csv


In [50]:
import pandas as pd
import re
from pathlib import Path

# 设置路径
file_location_path = Path.cwd()
project_base_path = file_location_path.parent.parent

# 加载已合并的CSV文件
csv_file = project_base_path / 'data' / 'processed' / 'landscape_ob' / 'filtered_objects2.csv'
data = pd.read_csv(csv_file)

# 检查列名是否正确
print(data.columns)

# 定义一个函数，去掉物体名称后面的置信度
def remove_confidence(objects_str):
    if isinstance(objects_str, str):  # 确保是字符串类型
        # 使用正则表达式匹配每个物体名称后面的 ':数字' 部分，并将其去掉
        return re.sub(r':\d+\.\d+', '', objects_str)
    else:
        return objects_str  # 如果不是字符串，直接返回原值

# 确保 'combined_objects' 列存在
if 'predicted_objects' in data.columns:
    # 对 'combined_objects' 列应用函数
    data['filtered_objects'] = data['predicted_objects'].apply(remove_confidence)

    # 重命名列 'combined_objects' 为 'object_type'，'filtered_objects' 为 'type_summary'
    data = data.rename(columns={
        'predicted_objects': 'object_type',
        'filtered_objects': 'type_summary'
    })

    # 保存处理后的数据为新CSV文件
    renamed_csv_file = project_base_path / 'data' / 'processed' / 'landscape_ob' / 'filtered_objects2.csv'  # 替换为保存路径
    data.to_csv(renamed_csv_file, index=False)

    print(f"重命名后的CSV文件已保存到: {renamed_csv_file}")
else:
    print("Error: 'combined_objects' 列不存在于数据中")


Index(['image_path', 'predicted_score', 'predicted_objects',
       'filtered_objects'],
      dtype='object')
重命名后的CSV文件已保存到: /home/ubuntu/landscape-aesthetics/data/processed/landscape_ob/filtered_objects2.csv
